## LangGraph - The Easy Way
https://www.youtube.com/watch?v=R8KB-Zcynxc&t=236s

In [242]:
# pip install python-dotenv
# pip install langgraph
!pip show python-dotenv langgraph langchain_openai

Name: python-dotenv
Version: 0.21.0
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: https://github.com/theskumar/python-dotenv
Author: Saurabh Kumar
Author-email: me+github@saurabh-kumar.com
License: BSD-3-Clause
Location: C:\Users\CGM\anaconda3\Lib\site-packages
Requires: 
Required-by: anaconda-cloud-auth, pydantic-settings
---
Name: langgraph
Version: 0.2.59
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: C:\Users\CGM\anaconda3\Lib\site-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-sdk
Required-by: 
---
Name: langchain-openai
Version: 0.2.12
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\CGM\anaconda3\Lib\site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


Nodes act like functions that can be called as needed. In our case Node 1 is our starting point and Node 2 is our finish point.

## 1. Simplest Graph

In [1]:
def function_1(input_1):
    return input_1 + " Hi "

def function_2(input_2):
    return input_2 + "there"

In [2]:
from langgraph.graph import Graph

# Creates an instance of the Graph class and assigns it to the variable
workflow1 = Graph()

# Adds two nodes to the graph
workflow1.add_node("node_1", function_1)
workflow1.add_node("node_2", function_2)

# Adds a directed edge from node_1 to node_2
workflow1.add_edge('node_1', 'node_2')

# Sets node_1 as the entry point of the workflow
workflow1.set_entry_point("node_1")
# Sets node_2 as the finish point of the workflow
workflow1.set_finish_point("node_2")

# Compiles the graph into an executable application or object
app1 = workflow1.compile()

# Executes the compiled workflow (app) with "Hello" as an input.
app1.invoke("Hello")

'Hello Hi there'

- `add_node(name, function)`: Adds a node to the graph. Each node represents a step in the workflow, and it’s linked to a specific callable function (e.g., function_1 or function_2).

- `add_edge(from_node, to_node)`: Establishes a directed connection between two nodes. This defines the execution order, ensuring to_node runs after from_node.

- `set_entry_point(node)`: Marks the starting node of the workflow. Execution begins here.

- `set_finish_point(node)`: Marks the final node of the workflow. Execution ends when this node completes.

- `compile()`: Converts the graph into an executable format, preparing the workflow for running the defined sequence of tasks.
- `invoke` method is typically used to trigger the workflow and pass any required input (in this case, the string "Hello").

In [3]:
input = 'Hello'
for output in app1.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'node_1':
---
Hello Hi 

---

Output from node 'node_2':
---
Hello Hi there

---



- `stream` method in the context of this code is used to <b>execute the workflow node by node and yield results progressively.</b>

## 2. Adding LLM Call

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
# Now you can access your environment variables using os.environ
os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY")

In [5]:
from langchain_openai import ChatOpenAI

# Set the model as ChatOpenAI
model = ChatOpenAI(temperature=0) 

#Call the model with a user message
model.invoke('Hey there')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cbbf06f4-6bc0-4534-88e0-ea629dcfdf69-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
model.invoke('Hey there').content

'Hello! How can I assist you today?'

- `.content:` An attribute of the response object that holds the main output (e.g., generated text or result).
- Use `invoke` when you want to <b>send data to the model, process it, and get a structured response back.</b>

In [7]:
def function_3(input_3):
    # Sends the input `input_1` to a model using the `invoke` method.
    # Retrieves the model's response and returns the `content` attribute of the response.
    response = model.invoke(input_3)
    return response.content

def function_4(input_4):
    # Prepends the string "Agent Says: " to the input `input_2` and returns the resulting string.
    return "Agent Says: " + input_4

1. Define a Langchain graph
2. Creates a new instance of a Langchain `Graph` object, which will represent the workflow structure.
3. Adds a node named "agent" to the workflow, associated with `function_3`.
4. This node will serve as the starting point and execute the logic defined in `function_3`.
5. Adds another node named "node_2", associated with `function_4`.
6. This represents the next step in the workflow after "agent".

In [8]:
workflow2 = Graph()

workflow2.add_node("agent", function_3)

workflow2.add_node("node_2", function_4)
workflow2.add_edge('agent', 'node_2')

workflow2.set_entry_point("agent")
workflow2.set_finish_point("node_2")

app2 = workflow2.compile()

In [9]:
app2.invoke("Hey there")

'Agent Says: Hello! How can I assist you today?'

In [10]:
input = 'Hey there'
for output in app2.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
Hello! How can I assist you today?

---

Output from node 'node_2':
---
Agent Says: Hello! How can I assist you today?

---



## 3. First functional Agent App - City Temperature
### Step 1: Parse the city mentioned

In [20]:
def function_5(input_5):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_5
    response = model.invoke(complete_query)
    return response.content

def function_6(input_6):
    return "Agent Says: " + input_6

In [21]:
# Define a Langchain graph
workflow3 = Graph()

#calling node 1 as agent
workflow3.add_node("agent", function_5)
workflow3.add_node("node_2", function_6)

workflow3.add_edge('agent', 'node_2')

workflow3.set_entry_point("agent")
workflow3.set_finish_point("node_2")

app3 = workflow3.compile()

In [22]:
app3.invoke("What's the temperature in Madrid")

'Agent Says: Madrid'

### Step 2: Adding a weather API call

In [23]:
!pip show pyowm

Name: pyowm
Version: 3.3.0
Summary: A Python wrapper around OpenWeatherMap web APIs
Home-page: https://github.com/csparpa/pyowm
Author: Claudio Sparpaglione
Author-email: csparpa@gmail.com
License: MIT
Location: C:\Users\CGM\anaconda3\Lib\site-packages
Requires: geojson, PySocks, requests
Required-by: 


In [24]:
from dotenv import load_dotenv
from langchain_community.utilities import OpenWeatherMapAPIWrapper
load_dotenv()
os.environ["OPENWEATHERMAP_API_KEY"] = os.environ.get("OPENWEATHERMAP_API_KEY")

weather = OpenWeatherMapAPIWrapper()

In [25]:
weather_data = weather.run("Seoul")
print(weather_data)

In Seoul, the current weather is as follows:
Detailed status: scattered clouds
Wind speed: 2.93 m/s, direction: 306°
Humidity: 46%
Temperature: 
  - Current: -1.69°C
  - High: -1.69°C
  - Low: -1.69°C
  - Feels like: -5.46°C
Rain: {}
Heat index: None
Cloud cover: 27%


In [26]:
def function_7(input_7):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_7
    response = model.invoke(complete_query)
    return response.content

def function_8(input_8):
    weather_data = weather.run(input_8)
    return weather_data

In [27]:
from langgraph.graph import Graph

workflow4 = Graph()

#calling node 1 as agent
workflow4.add_node("agent", function_7)
workflow4.add_node("tool", function_8)

workflow4.add_edge('agent', 'tool')

workflow4.set_entry_point("agent")
workflow4.set_finish_point("tool")

app4 = workflow4.compile()

In [28]:
app4.invoke("What's the temperature in Helsinki")

'In Helsinki, the current weather is as follows:\nDetailed status: mist\nWind speed: 4.02 m/s, direction: 74°\nHumidity: 97%\nTemperature: \n  - Current: 1.44°C\n  - High: 2.65°C\n  - Low: 0.25°C\n  - Feels like: -2.53°C\nRain: {}\nHeat index: None\nCloud cover: 75%'

In [29]:
input = "What's the temperature in Stockholm"
for output in app4.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
Stockholm

---

Output from node 'tool':
---
In Stockholm, the current weather is as follows:
Detailed status: mist
Wind speed: 3.6 m/s, direction: 250°
Humidity: 98%
Temperature: 
  - Current: 5.04°C
  - High: 5.59°C
  - Low: 4.86°C
  - Feels like: 2.15°C
Rain: {}
Heat index: None
Cloud cover: 75%

---



### Step 3 Adding another LLM Call to filter results

In [30]:
def function_11(input_3):
    complete_query = "Your task is to provide info concisely based on the user query. Following is the user query: " + "user input"
    response = model.invoke(complete_query)
    return response.content

In [31]:
# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
AgentState["messages"] = []

In [32]:
AgentState

{'messages': []}

In [33]:
def function_9(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = "Your task is to provide only the city name based on the user query. \
                    Nothing more, just the city name mentioned. Following is the user query: " + user_input
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def function_10(state):
    messages = state['messages']
    agent_response = messages[-1]
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(agent_response)
    state['messages'].append(weather_data)
    return state

def function_11(state):
    messages = state['messages']
    user_input = messages[0]
    available_info = messages[-1]
    agent2_query = "Your task is to provide concisely the current temperature based on the available information. \
                    Do not include any other details. Following is the user query: " + user_input + \
                    " Available information: " + available_info
    response = model.invoke(agent2_query)
    return response.content

In [34]:
from langgraph.graph import Graph

workflow5 = Graph()

workflow5.add_node("agent", function_9)
workflow5.add_node("tool", function_10)
workflow5.add_node("responder", function_11)

workflow5.add_edge('agent', 'tool')
workflow5.add_edge('tool', 'responder')

workflow5.set_entry_point("agent")
workflow5.set_finish_point("responder")

app5 = workflow5.compile()

In [35]:
inputs = {"messages": ["what is the temperature in Singapore"]}
app5.invoke(inputs)

'The current temperature in Singapore is 29.15°C.'

In [36]:
input = {"messages": ["what is the temperature in Singapore"]}
for output in app5.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': ['what is the temperature in Singapore', 'Singapore']}

---

Output from node 'tool':
---
{'messages': ['what is the temperature in Singapore', 'Singapore', 'In Singapore, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 6.69 m/s, direction: 20°\nHumidity: 68%\nTemperature: \n  - Current: 29.15°C\n  - High: 29.97°C\n  - Low: 27.95°C\n  - Feels like: 32.68°C\nRain: {}\nHeat index: None\nCloud cover: 75%']}

---

Output from node 'responder':
---
The current temperature in Singapore is 29.15°C.

---



In [38]:
input2 = {"messages": ["How are you today?"]}
app5.invoke(input2)

'The current temperature in Istanbul is 8.68°C.'

### Step 4 AgentState
Manages an agent's state by storing <b>conversation history</b> and <b>workflow context</b> in a `structured dictionary`, enabling <b>dynamic decision-making</b> and <b>coherent interactions</b> in chatbots and automated systems

1. `TypedDict`: Used to create structured dictionary-like classes with predefined keys and types.
2. `Annotated`: Allows attaching metadata or context to a type.
3. `Sequence`: Represents a generic ordered collection, such as a list or tuple.
4. `operator`: Provides functional equivalents of built-in operators, like addition, subtraction, etc.
5. `BaseMessage`: Represents a fundamental message structure used in the LangChain framework (It is likely used for storing or processing individual messages in a workflow or conversation.)

In [42]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

1. `AgentState` define a structured dictionary class called AgentState
2. It inherits from `TypedDict`, which allows specifying fixed keys and their associated types.
3. Define the key `messages`:
    - It is a sequence of `BaseMessage` objects (e.g., a list or tuple).
    - The `Annotated` type wraps the sequence type to include additional metadata.
    - `operator.add` is used as metadata, indicating that this sequence can be combined using addition (concatenation).

In [43]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

1. `convert_to_openai_function` to make tools compatible with OpenAI's API.
2. `OpenWeatherMapQueryRun` a prebuilt tool to fetch weather data

In [44]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun

1. `tools` initializes a list of tools, specifically the OpenWeatherMapQueryRun tool.
2. The tool interacts with the OpenWeatherMap API to fetch weather data, such as temperature, humidity, etc.
3. Additional tools can be added to this list as needed.
4. `model` initializes an instance of the ChatOpenAI model with specific parameters:
   - `streaming=True`: Enables streaming responses, allowing partial outputs to
6. `functions` converts each tool in the `tools` list into an OpenAI-compatible function.
   - `tools`: A list of tool instances, such as APIs or custom functions.
   - `convert_to_openai_function(t)`: A utility that wraps each tool to make it usable with OpenAI's function-calling API.
7. `model` binds the initialized tools (e.g., OpenWeatherMapQueryRun) to the ChatOpenAI model.
   - This enables the model to invoke these tools during its operation, enhancing its ability to fetch real-time weather data.
   - The model can now use the tool(s) dynamically based on the conversation or function call requirements.
   - `bind_tools` links external tools (e.g., APIs or functions) to the OpenAI model

In [45]:
tools = [OpenWeatherMapQueryRun()]
model = ChatOpenAI(temperature=0, streaming=True)
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_tools(tools)

1. This function processes the given `state`, which is expected to be a dictionary.
2. The state contains a key `messages` that holds a sequence of conversation messages.
3. `messages` extracts the messages from the state. These represent the conversation context so far.
4. `response` sends the messages to the `model` using its `invoke` method.
   - The `model` processes the messages (e.g., user input or context) and generates a response.
   - The response is likely a new message generated by the AI model.
5. Returns a new state with the updated `messages`.
   - The new state contains only the response in its `messages` key.
   - This could be useful for resetting or isolating responses in a workflow.

In [46]:
def function_12(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

1. `ToolNode` is used to integrate and execute tools within a LangGraph workflow.
   - It allows defining nodes in the graph that can perform specific tasks using external tools.
2. `FunctionMessage` represents a message generated by a tool or function in a workflow.
   - It can store the content of the message and metadata like the function name.

In [1]:
from langgraph.prebuilt import ToolNode
import json
from langchain_core.messages import FunctionMessage
from IPython.display import Image, display

1. Creates a new ToolNode instance and assigns it to the variable "tool_node".
   - `ToolNode`: A class from LangGraph used to define nodes in a workflow that execute tools.
   - `tools`: A list or collection of tool instances (e.g., API integrations or custom functions).
   - These tools are registered with the ToolNode, enabling the node to invoke them during execution.
2. "function_13" processes the current agent state to extract the latest message,
   - invokes a tool based on the message's details, and updates the state with the tool's response.
3. `messages` extracts the list of messages from the current state.
   - `messages` contains the conversation history or workflow messages.
4. "last_message" retrieves the last message in the conversation.
   - This is assumed to contain the query or function call details that need to be sent to the tool.
5. "parsed_tool_input" parses the arguments from the last message's `function_call`.
   - The arguments are expected to be in JSON format, so they are decoded using `json.loads`.
6. "action" constructs a `ToolInvocation` object with the tool name and the specific input extracted from the message.
   - `tool`: The name of the tool to be invoked, extracted from the function call metadata.
   - `tool_input`: The actual input for the tool, retrieved from the parsed arguments.
7. "response" calls the tool using the ToolNode (`tool_node`) and retrieves its response.
8. "function_message" creates a new FunctionMessage to encapsulate the tool's response.
   - `content`: The response from the tool, converted to a string.
   - `name`: The name of the tool that generated the response.
9. Returns the updated state containing the new message (the tool's response).
   - This ensures the workflow can continue with the updated information.

In [48]:
tool_node = ToolNode(tools)

def function_13(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent
    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input['__arg1'],
    )
    response = tool_node.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {"messages": [function_message]}

1. "where_to_go" determines the next step in a workflow based on the state of the conversation.
   - Specifically, it checks if the last message contains a "function_call" instruction.
2. "messages" extracts the list of messages from the current state.
   - These messages represent the conversation history or the workflow context.
3. "last_message" retrieves the last message in the conversation.
   - This message contains the latest user query or system response.
4. The "if function" checks if the last message contains a "function_call" in its additional metadata (kwargs).
   - This indicates that a function/tool needs to be executed based on the message.

In [49]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    if "function_call" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

1. `StateGraph`: A class used to define and manage workflows or state-based transitions in LangGraph.
   - It provides a way to build graphs where each node represents a state or function in the workflow.
2. `END`: A special constant used to denote the end point of the graph or workflow. It signals where the process should terminate.

In [50]:
from langgraph.graph import StateGraph, END

1. "workflow6" initializes a new workflow using the StateGraph class.
2. `StateGraph`: A graph-based framework for defining workflows or processes, where each node represents a state or function.
3. `AgentState`: The structured dictionary that represents the state of the agent, including its messages and other relevant data.
4. <b>`add_conditional_edges`</b> requires the following info below.
   - First, we define the start node. We use `agent`.
   - This means these are the edges taken after the `agent` node is called.
   - Next, we pass in the function that will determine which node is called next, in our case where_to_go().

In [51]:
workflow6 = StateGraph(AgentState)

workflow6.add_node("agent", function_12)
workflow6.add_node("tool", function_13)

workflow6.add_conditional_edges("agent", where_to_go,{  # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow6.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.
workflow6.set_entry_point("agent")

app6 = workflow6.compile()

1. Imports the `HumanMessage` class from the LangChain Core module.
   - `HumanMessage`: Represents a message sent by a human user in a conversation or workflow.
   - This is part of the LangChain framework's messaging system, which structures and manages interactions between users, agents, and tools.

In [52]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the temperature in Seoul")]}
app6.invoke(inputs)

{'messages': [HumanMessage(content='what is the temperature in Seoul', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Ze6HNUEmYSRKKGKt5S2qlyks', 'function': {'arguments': '{"location":"Seoul"}', 'name': 'open_weather_map'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-8a5088a2-393c-4d73-9130-897d263a1aac-0', tool_calls=[{'name': 'open_weather_map', 'args': {'location': 'Seoul'}, 'id': 'call_Ze6HNUEmYSRKKGKt5S2qlyks', 'type': 'tool_call'}])]}

In [391]:
inputs = {"messages": [HumanMessage(content="what is the temperature in Seoul")]}
for output in app6.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_iQfug9LwSS86TYmX0GfxbWfR', 'function': {'arguments': '{"location":"Seoul"}', 'name': 'open_weather_map'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-8ad458b5-156d-45c6-8f3f-66be6a63e1e7-0', tool_calls=[{'name': 'open_weather_map', 'args': {'location': 'Seoul'}, 'id': 'call_iQfug9LwSS86TYmX0GfxbWfR', 'type': 'tool_call'}])]}

---

